<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Biomass gridding

_by Joe Hamman (CarbonPlan), September 29, 2020_

This notebook uses Verde to grid biomass point estimates.


In [ ]:
import pandas as pd
import pyproj
import xarray as xr
from carbonplan.forests.fit.interp import points_to_grid
import zarr
import os

## get the input data


In [ ]:
# open points data
data = pd.read_parquet("/home/jovyan/interpolation_test_raw_data.parquet")

# create mask
classes = [41, 42, 43, 51, 52, 90]
bands = xr.concat(
    [
        xr.open_rasterio(
            f"https://carbonplan.blob.core.windows.net/carbonplan-data/processed/nlcd/conus/4000m/2016_c{c}.tif"
        )
        for c in classes
    ],
    dim=xr.Variable("band", classes),
)
mask = bands.sum("band", keep_attrs=True)
mask.plot(cmap="Greys")

## grid the data


In [ ]:
gridded_points = points_to_grid(data, mask)
gridded_points.plot(cmap="Greens", vmax=400)

## save the gridded data as a zarr store


In [ ]:
store = zarr.storage.ABSStore(
    "carbonplan-scratch",
    prefix="biomass_grid.zarr",
    account_name="carbonplan",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
)

In [ ]:
gridded_points.to_dataset(name="biomass").to_zarr(
    store, mode="w", consolidated=True
)